In [1]:
import os
from skimage import io
from skimage.metrics import structural_similarity as ssim
import numpy as np
from PIL import Image

In [2]:
# Paths to folders containing images
real_images_folder = 'C:/Users/Arjit/OneDrive/Desktop/MTP/CODE/Color Diffusion/Color-diffusion/test_images/100_gt'
fake_images_folder = 'C:/Users/Arjit/OneDrive/Desktop/MTP/CODE/Color Diffusion/Color-diffusion/results/100_output'

In [3]:
# Function to calculate SSIM score
def calculate_ssim(real_image_path, fake_image_path):
    # Open images and convert to grayscale (if needed)
    real_image = Image.open(real_image_path).convert('RGB')
    fake_image = Image.open(fake_image_path).convert('RGB')
    
    # Resize images to be the same size
    real_image = real_image.resize((fake_image.width, fake_image.height))
    
    # Convert images to numpy arrays
    real_image = np.array(real_image)
    fake_image = np.array(fake_image)
    
    # Compute SSIM between the two images
    score, _ = ssim(real_image, fake_image, full=True, multichannel=True)
    return score

In [4]:
real_image_files = sorted(os.listdir(real_images_folder))
fake_image_files = sorted(os.listdir(fake_images_folder))

In [5]:
# List to store SSIM scores
ssim_scores = []

# Loop through each image pair and calculate SSIM
for real_file, fake_file in zip(real_image_files, fake_image_files):
    real_image_path = os.path.join(real_images_folder, real_file)
    fake_image_path = os.path.join(fake_images_folder, fake_file)
    
    # Calculate SSIM score
    score = calculate_ssim(real_image_path, fake_image_path)
    ssim_scores.append(score)

In [6]:
# Calculate the average SSIM score
average_ssim = np.mean(ssim_scores)
print(f"Average SSIM Score: {average_ssim}")


Average SSIM Score: 0.8389581615659553


In [18]:
import os
from skimage.io import imread
from skimage.metrics import structural_similarity as ssim
import numpy as np
from tqdm import tqdm

In [22]:
def preprocess_image(image_path, size=(256, 256)):
    img = imread(image_path)
    # Convert grayscale to RGB
    if img.ndim == 2:
        img = np.stack([img] * 3, axis=-1)
    elif img.shape[2] == 4:
        img = img[:, :, :3]  # Remove alpha channel

    # Resize to target size and normalize to [0, 1]
    img_resized = resize(img, size, anti_aliasing=True)
    return img_resized

def calculate_ssim(img1, img2):
    score, _ = ssim(img1, img2, channel_axis=-1, full=True)
    return score

def compute_folder_ssim(folder1, folder2, size=(256, 256)):
    files1 = sorted([f for f in os.listdir(folder1) if f.lower().endswith(('png', 'jpg', 'jpeg'))])
    files2 = sorted([f for f in os.listdir(folder2) if f.lower().endswith(('png', 'jpg', 'jpeg'))])

    ssim_scores = []

    for f1, f2 in tqdm(zip(files1, files2), total=min(len(files1), len(files2))):
        path1 = os.path.join(folder1, f1)
        path2 = os.path.join(folder2, f2)
        try:
            img1 = preprocess_image(path1, size)
            img2 = preprocess_image(path2, size)
            score = calculate_ssim(img1, img2)
            ssim_scores.append(score)
        except Exception as e:
            continue

    if ssim_scores:
        return np.mean(ssim_scores)
    return None

In [23]:
# Paths to folders containing images
folder_real = '/home/satyanshu/Documents/test/text-guided-image-colorization/example/test_gt'
folder_fake = '/home/satyanshu/Documents/test/text-guided-image-colorization/example/test_output/results/colorized'

In [24]:

avg_ssim = compute_folder_ssim(folder_real, folder_fake)
print("Average SSIM:", avg_ssim)


100%|██████████| 1000/1000 [00:03<00:00, 293.17it/s]

Average SSIM: None
